## Programming for Data Science



### **Project 1**
**Group members:**
* Ghita Rholi, 20221491@novaims.unl.pt
* Mariana Ferreira, 20211637@novaims.unl.pt
* Mariana Neto, 20211527@novaims.unl.pt



### Step 3. Feature extraction

Extraction of potential useful features and assessment of features correlation structure.
- **1.** Totals
    - **1.1** Total amount of money spent
    - **1.2** Total amount of purchases
- **2.** Percentages
    - **2.1** Percentage of purchases
    - **2.2** Percentage of the amount of money spent
- **3.** Child-related features
    - **3.1** Number of Children
    - **3.2** Has Child
- **4.** NaN/Infinite Values
- **5.** Redundancy analysis
- **6.** Exporting the dataset
   
<br>

##### Importing the necessary libraries

In [1]:
import numpy as np

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

# Imports spearmanr function from scipy library
from scipy.stats import spearmanr, pearsonr

import warnings
warnings.filterwarnings('ignore')

<br>

##### Importing the dataset from the previous notebook

In [2]:
data = pd.read_excel("after_preprocess.xlsx", index_col = "ID")

<br>

## Feature extraction

Feature extraction is the process of defining a set of (new) features, which are a linear combination of the existing data. 
By eliminating redundant data from a dataset, feature extraction helps to effectively represent crucial information for future analyses.

Having familiarized ourselves with the dataset in the previous steps, it is now easier to look at the data and identify new possible variables that might have a good impact on the model. To do that we can start by looking at the dataset and visualzing all the columns one more time, using the ``.T`` method on the 'data'.

In [3]:
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


Prior to generating the new columns, we create a function (``get_columns_index()``) that will be useful for visual purposes. That is, this function will return either a text where the index of a given column (or a list of columns) is described or it will return the index itself. If the parameter index_describe is 'yes', then the function will print the column name and the corresponding index, and if it is 'no', the function will retun the index value.
This function is defined in the code below.


In [4]:
def get_columns_index(column_names, index_described):
    # Column names can either be a list or a string
    # If column_names is a list
    if isinstance(column_names, list):
        # It uses all the column names in the list
        for column_name in column_names:
            # It gets the index of the columns corresponding to the names
            index = data.columns.get_loc(column_name)
            # If it is wanted the index to be described, index_describe is equal to yes
            if index_described == 'yes':
                # It prints the column name and the associated index
                print(f'Index of column named {column_name} is: {index}')
            # If it is not wanted the index to be described, index_describe is equal to no
            elif index_described == 'no':
                return index
            # If the value of index_described is not valid, nothing happens
            else:
                return
    # If column_names is a string
    elif isinstance(column_names, str):
        # It gets the index of the column corresponding to the name
        index = data.columns.get_loc(column_names)
        if index_described == 'yes':
            # It prints the column name and the associated index
            print(f'Index of column named {column_names} is: {index}')
            # If it is not wanted the index to be described, index_describe is equal to no
        elif index_described == 'no':
            return index
            # If the value of index_described is not valid, nothing happens
        else:
            return
    # If column_names is neither a string nor a list, nothing happens
    else:
        return

<br>
<br>

## **1.** Totals

The Totals features correspond to the sum of all the features within a certain group. This linear combination allows for a more accurate interpretation of an individual's behavior, since it provides the overall behavior in a category (Which, in this case, will be purchases and money spent).

To automatize the extraction of such variables, a function (``total()``) is created. This function is defined in the code below.

In [5]:
def total(columns, suffix, index):
    total = 0
    for column in columns:
        total += data[column]
    data.insert(loc=index, column=f"Total{suffix}", value = total)

<br>

#### **1.1** Total amount of money spent

The Total amount of money spent (TotalMnt), corresponding to the sum of the columns MntFishProducts, MntMeatProducts, MntFruits, MntSweetProducts, and MntWines, is an important measurement. This is due to the fact that, in this particular model, it may be relevant to know which individuals spent the most and the least money within the company.

Just before creating this feature, the function ``get_columns_index()`` is used so that it is known the indexes of the columns used to generate the respective feature. With that information, it is possible to place the new feature next to the ones it is related to.

In [6]:
column_names = ["MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts"]
get_columns_index(column_names, 'yes')

Index of column named MntWines is: 7
Index of column named MntFruits is: 8
Index of column named MntMeatProducts is: 9
Index of column named MntFishProducts is: 10
Index of column named MntSweetProducts is: 11


The code above enabled the acknowlegment that the last column related to the amount of money spent on products is in the index 12, therefore, the feature TotalMnt may be placed in index 13. In the cell below, the new column is inserted in the data, in index 13, with the suffix 'Mnt' and considering all the column names in the column_names list from the code above.

The data is being displayed, transposed, so that we can check if the position is correct.

In [7]:
total(column_names, 'Mnt', 13)
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


Through the visualization of the table above, it is observed that the new column (TotalMnt) is in the intended position.

The ``.filter()`` function subsets the dataframe columns according to the word "Mnt", in order to get a better look at the values related to wines, fruits, meat, fish, sweet, gold products and the total amount spent.

In [8]:
data.filter(regex="Mnt",axis=1)

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,TotalMnt
ID,,,,,,,
5314,270,3,27,39,6,99,345
2493,356,0,107,19,9,117,491
4931,1,0,1725,2,1,1,1729
1100,24,4,22,0,2,9,52
7224,10,0,10,3,1,7,24
...,...,...,...,...,...,...,...
3887,3,17,26,20,1,39,67
5181,2,4,2,0,11,5,19
5758,1074,0,69,0,0,46,1143


With the table above it is possible to observe that the TotalMnt column was correctly implemented.

<br> 

#### **1.2** Total amount of purchases

The Total amount of Purchases (TotalPurchases) corresponds to the sum of the amount of purchases made using the companies' website (NWebPurchases), the catalog (NCatalogPurchases) and in the store (NStorePurchases). This feature may help understand which customers have purchased the most and the least.

The function below is used to get the indexes of the columns NWebPurchases, NCatalogPurchases and NStorePurchases.

In [9]:
column_names = ["NWebPurchases","NCatalogPurchases","NStorePurchases"]
get_columns_index(column_names, 'yes')

Index of column named NWebPurchases is: 17
Index of column named NCatalogPurchases is: 18
Index of column named NStorePurchases is: 19


The last index given by the function is 19. Therefore, for the new feature to be placed next to the ones it is related to, it will be stored at index 20.

In [10]:
total(column_names, 'Purchases', 20)
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


<br>

We can use, again, the ``.filter()`` method to get a better look at the features related to purchases.

In [11]:
data.filter(regex="Purchases",axis=1)

,NDealsPurchases,NWebPurchases,NCatalogPurchases,NStorePurchases,TotalPurchases
ID,,,,,
5314,7,7,1,5,13
2493,2,7,1,8,16
4931,0,0,28,0,28
1100,4,3,1,3,7
7224,2,1,1,2,4
...,...,...,...,...,...
3887,2,2,0,3,5
5181,1,1,0,2,3
5758,1,10,4,13,27


With the table above it is possible to observe that the TotalPurchases column was correctly implemented.

<br>
<br>

## **2.** Percentages

Percentages are a commonly used way to represent data. It is easy to understand in any context, especially in comparison to mere numbers that might not convey anything to potential readers. Thus, it could be beneficial to create new features that will represent some useful percentage for the model.

To automatize the creation of this new features, it is first defined a function (``percentages()``) which is defined in the code below.

In [12]:
def percentages(numerator, denominator, index, suffix):
    percentages = (data[numerator] / data[denominator])*100
    data.insert(loc = index, column= 'P' + suffix, value = percentages)

<br>

#### **2.1** Percentage of purchases
An interesting feature that can help understand which purchasing channel is most appealing to a customer is the percentage of purchases. This will allow insight on the purchases made through the catalog, the company's website, and the store in such a way as to recognize patterns that indicate a customer's preference (purchasing from the store, website, or catalog). 

<br>

**2.1.1** Percentage of catalog purchases

The percentage of catalog purchases is the proportion of the amount of purchases made using a catalog, by the total number of purchases made using the three different channels. 

In the next cell it is executed the function that returns the index of the column NCatalogPurchases.

In [13]:
get_columns_index("NCatalogPurchases", 'yes')

Index of column named NCatalogPurchases is: 18


NCatalogPurchases is at index 18, therefore, the feature PCatalogPurchases will be store at index 19.

The following code creates a feature with the mentioned percentage of NCatalogPurchases in TotalPurchases, which is placed in index 19.

In [14]:
percentages('NCatalogPurchases', 'TotalPurchases', 19, 'CatalogPurchases')
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


This ``filter()`` method generates a dataframe that exposes the relative and absolute values of purchases made using a catalog. 

In [15]:
data.filter(regex='NCatalogPurchases|PCatalogPurchases',axis=1)

,NCatalogPurchases,PCatalogPurchases
ID,,
5314,1,7.692308
2493,1,6.250000
4931,28,100.000000
1100,1,14.285714
7224,1,25.000000
...,...,...
3887,0,0.000000
5181,0,0.000000
5758,4,14.814815


<br>

**2.1.2** Percentage of Web purchases

The percentage of Web purchases is the proportion of the amount of purchases made using the Website, by the total number of purchases made using the three different channels. 

In the next cell it is executed the function that returns the index of the column NWebPurchases.

In [16]:
get_columns_index("NWebPurchases", 'yes')

Index of column named NWebPurchases is: 17


NWebPurchases is at index 17, therefore, the feature PWebPurchases will be store at index 18.

The following code creates a feature with the mentioned percentage of NWebPurchases in TotalPurchases, which is placed in index 18.

In [17]:
percentages('NWebPurchases', 'TotalPurchases', 18, 'WebPurchases')
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


This ``filter()`` method generates a dataframe that exposes the relative and absolute values purchases made using the website. 

In [18]:
data.filter(regex='NWebPurchases|PWebPurchases',axis=1)

,NWebPurchases,PWebPurchases
ID,,
5314,7,53.846154
2493,7,43.750000
4931,0,0.000000
1100,3,42.857143
7224,1,25.000000
...,...,...
3887,2,40.000000
5181,1,33.333333
5758,10,37.037037


<br>

**2.1.3** Percentage of Store purchases

The percentage of Store purchases is the proportion of the amount of purchases made in a store, by the total number of purchases made using the three different channels. 

In the next cell it is executed the function that returns the index of the column NStorePurchases.

In [19]:
get_columns_index("NStorePurchases", 'yes')

Index of column named NStorePurchases is: 21


NStorePurchases is at index 21, therefore, the feature PStorePurchases will be store at index 22.

The following code creates a feature with the mentioned percentage of NStorePurchases in TotalPurchases, which is placed in index 22.

In [20]:
percentages('NStorePurchases', 'TotalPurchases', 22, 'StorePurchases')
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
MntFruits,3,0,0,4,0,0,0,0,4,15,...,81,1,29,0,6,17,4,0,10,0
MntMeatProducts,27,107,1725,22,10,3,0,2,1,287,...,149,11,514,275,2,26,2,69,253,108


This ``filter()`` method generates a dataframe that exposes the relative and absolute values of purchases made in a store. 

In [21]:
data.filter(regex='NStorePurchases|PStorePurchases',axis=1)

,NStorePurchases,PStorePurchases
ID,,
5314,5,38.461538
2493,8,50.000000
4931,0,0.000000
1100,3,42.857143
7224,2,50.000000
...,...,...
3887,3,60.000000
5181,2,66.666667
5758,13,48.148148


<br>

#### **2.2** Percentage of the amount of money spent

A useful feature that can assist in understanding on which product the largest and smallest amount of money spent is, is the percentage of money spent. This will allow us to know the proportion of money spent on each product.

A similar process necessary to accomplish the creation of all the features corresponding to the amount of money spent was already executed for the percentage of purchases, step by step. Hence, we can now do the automatize that process. To do that, it is executed in the code below a for loop that will create a percentage feature for every Mnt column relatively to the TotalMnt.

In [22]:
MntColumns = ["MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts",'MntGoldProds']
# Use each Mnt column
for column in MntColumns:
    # Get the index of the column to place its percentage next to it
    index = get_columns_index(column, 'no')
    # Create the percentage feature, and place it an index after the column it represents
    percentages(column, 'TotalMnt', index + 1, column)
data.filter(regex ='Mnt')

,MntWines,PMntWines,MntFruits,PMntFruits,MntMeatProducts,PMntMeatProducts,MntFishProducts,PMntFishProducts,MntSweetProducts,PMntSweetProducts,MntGoldProds,PMntGoldProds,TotalMnt
ID,,,,,,,,,,,,,
5314,270,78.260870,3,0.869565,27,7.826087,39,11.304348,6,1.739130,99,28.695652,345
2493,356,72.505092,0,0.000000,107,21.792261,19,3.869654,9,1.832994,117,23.828921,491
4931,1,0.057837,0,0.000000,1725,99.768652,2,0.115674,1,0.057837,1,0.057837,1729
1100,24,46.153846,4,7.692308,22,42.307692,0,0.000000,2,3.846154,9,17.307692,52
7224,10,41.666667,0,0.000000,10,41.666667,3,12.500000,1,4.166667,7,29.166667,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3887,3,4.477612,17,25.373134,26,38.805970,20,29.850746,1,1.492537,39,58.208955,67
5181,2,10.526316,4,21.052632,2,10.526316,0,0.000000,11,57.894737,5,26.315789,19
5758,1074,93.963255,0,0.000000,69,6.036745,0,0.000000,0,0.000000,46,4.024497,1143


Upon viewing the table above, it is clear that the generation of the new variables was successful.   

<br><br>

### **3.** Child-related features

Two new features will be created to have a better overview of the costumer's family situation. This can help, for instance, knowing the influence of having kids, what type of products the costumer buys and if it affects the decision making when deciding to accept the offer of the campaign or not.


#### **3.1** Number of Children

The cell bellow creates the feature NChildren, which is the sum of the number of kids and teenagers a costumer has. To create this feature, it can be used a modified version (``create_column``) of the function ``total()``.

In [23]:
# Modified version of total(), the modification is in the last row of the function, with column = name
def create_column(columns, name, index):
    total = 0
    for column in columns:
        total += data[column]
    # The name of the new column is given as a parameter of the function by name
    data.insert(loc=index, column= name, value = total)

# Getting the index of the column NTeens
index = get_columns_index('NTeens', 'no')

# Creating NChildren
create_column(['NKids', 'NTeens'], 'NChildren', index+1)

# Displaying the transposed version of the data
data.T


ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
NChildren,1,1,0,3,0,3,0,1,0,3,...,0,2,0,0,1,1,1,0,2,2
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483
PMntWines,78.26087,72.505092,0.057837,46.153846,41.666667,62.5,NaN,94.736842,30.0,79.88757,...,38.556701,66.666667,59.115282,68.452895,36.363636,4.477612,10.526316,93.963255,34.30127,80.904523


<br>

#### **3.2** Has Child

The second child-related feature is HasChild, which represents the customers that have at least one child. That is, if an individual has a kid or a teen, its corresponding HasChild is equal to 1. If one indivual does not have children, HasChild is equal to 0. 

In the code below, it is created the above-mentioned feature.

In [24]:
# Getting the values for HasChild
HasChild = np.where(data.NChildren > 0, 1, 0)
# Getting the index of NChildren, the last column about children in the dataset
index = get_columns_index('NChildren', 'no')
# Adding the column to the dataset
data.insert(loc=index+1, column= 'HasChild', value = HasChild)
# Displaying the data transposed
data.T

ID,5314,2493,4931,1100,7224,2795,5555,10277,624,9499,...,9937,2461,9606,7214,6424,3887,5181,5758,9729,10478
Education,Graduation,Graduation,Graduation,Master,Graduation,Master,Graduation,Graduation,Master,Graduation,...,Graduation,Master,PhD,Graduation,Graduation,Graduation,Basic,Graduation,Graduation,PhD
Age,67,52,41,58,41,60,43,57,34,64,...,39,63,53,61,41,48,36,36,63,68
CustomerDays,1750,1613,1708,1379,1464,1645,1424,1775,1878,1700,...,1460,1475,1588,1641,1432,1877,1748,1448,1674,1922
MaritalStatus,Single,Married,Single,Single,Married,Single,Divorced,Single,Single,Married,...,Single,Single,Married,Married,Single,Single,Single,Married,Widow,Married
NKids,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,1,1,1,0,1,1
NTeens,1,1,0,2,0,1,0,1,0,2,...,0,1,0,0,0,0,0,0,1,1
NChildren,1,1,0,3,0,3,0,1,0,3,...,0,2,0,0,1,1,1,0,2,2
HasChild,1,1,0,1,0,1,0,1,0,1,...,0,1,0,0,1,1,1,0,1,1
Income,40689.0,61286.0,157146.0,41275.0,31353.0,30523.0,153924.0,30081.0,18890.0,93404.0,...,70337.0,51124.0,69969.0,62187.0,33178.0,27242.0,24367.0,65169.0,58275.0,55517.0
MntWines,270,356,1,24,10,5,0,36,6,1279,...,187,26,882,792,12,3,2,1074,189,483


<br>

### **4.** NaN/Infinite Values

Some of the extracted features, namely the precentages, contain NaN/Infinite values, given the fact that some denominators are 0. The cell bellow returns the number of NaN values each feature has, using the ``.isin()`` method.

In [25]:
# List containing the columns with the NaN values
columns = ["PCatalogPurchases", "PWebPurchases", "PStorePurchases", "PMntWines","PMntFruits","PMntMeatProducts","PMntFishProducts","PMntSweetProducts",'PMntGoldProds']
# isin() checks whether there is NaN/Infinite values and infinite values (either positive and negative) in columns
print(data[columns].isin([np.nan, np.inf, -np.inf]).sum())

PCatalogPurchases    6
PWebPurchases        6
PStorePurchases      6
PMntWines            6
PMntFruits           6
PMntMeatProducts     6
PMntFishProducts     6
PMntSweetProducts    6
PMntGoldProds        6
dtype: int64


The number of missing values are equal to 6 in each column, since it is a very small number (regarding the size of the dataset), the NaN/Infinite values can be replace with the mean value of each variable. The following cell replaces the NaN/Infinite values and checks if there is any after replacement, using the ``.isin()`` method.


In [26]:
for missingvalues in columns:
    data[missingvalues].replace(to_replace=[np.nan, np.inf, -np.inf], value=data[missingvalues].mean(), inplace=True)
    print("Is there any NaN or Infinite value in {} after replacement? R:".format(missingvalues),
      data[missingvalues].isin([np.nan, np.inf, -np.inf]).any())

Is there any NaN or Infinite value in PCatalogPurchases after replacement? R: False
Is there any NaN or Infinite value in PWebPurchases after replacement? R: False
Is there any NaN or Infinite value in PStorePurchases after replacement? R: False
Is there any NaN or Infinite value in PMntWines after replacement? R: False
Is there any NaN or Infinite value in PMntFruits after replacement? R: False
Is there any NaN or Infinite value in PMntMeatProducts after replacement? R: False
Is there any NaN or Infinite value in PMntFishProducts after replacement? R: False
Is there any NaN or Infinite value in PMntSweetProducts after replacement? R: False
Is there any NaN or Infinite value in PMntGoldProds after replacement? R: True


<br>

### **5.** Redundancy analysis

A statistical technique for evaluating the redundancy existent in a datset is correlation analysis. This will identify which variables are highly correlation between each other and therefore explain each others behaviour. The columns that have that form of correlation can be revealed as unecessary. This process be done using ``.corr()`` method, which is able to exclude null values.

First, the columns for which the correlation will be calculated are chosen (numerical columns). In the cell below, a list with all these columns is created.

In [27]:
# Numerical columns' names of the 'data' dataset stored in the 'numerical_columns' array
numerical_columns = data.select_dtypes(include = "number").columns
# Converting the array 'numerical_columns' into a list
numerical_columns = list(numerical_columns)
# Removing the element 'Response' from the list
numerical_columns.remove('Response')
# Displaying the list
print(numerical_columns)

['Age', 'CustomerDays', 'NKids', 'NTeens', 'NChildren', 'HasChild', 'Income', 'MntWines', 'PMntWines', 'MntFruits', 'PMntFruits', 'MntMeatProducts', 'PMntMeatProducts', 'MntFishProducts', 'PMntFishProducts', 'MntSweetProducts', 'PMntSweetProducts', 'MntGoldProds', 'PMntGoldProds', 'TotalMnt', 'Complain', 'NDealsPurchases', 'NWebPurchases', 'PWebPurchases', 'NCatalogPurchases', 'PCatalogPurchases', 'NStorePurchases', 'PStorePurchases', 'TotalPurchases', 'NWebVisitsMonth', 'Response1', 'Response2', 'Response3', 'Response4', 'Response5']


Afterwards, it is used the spearman's coefficient to calculate the correlation between the features stored in the list above. In this case the spearman's coefficient was choosen instead of the pearson's due to its properties. That is, whereas the pearson's can only calculate linear correlations, the spearman's coefficient can not only calculate those types of correlations, but also the non-linear ones. Therefore, it can work with non-numeric features. 

In the code below, the correlation matrix is being computed and displayed.

In [28]:
# Computing the correlation matrix, using spearman's coefficient between all the numerical features
plcc = data.loc[:, numerical_columns].corr(method="spearman")

# Displays the correlation matrix
plcc.style.background_gradient(cmap='YlOrRd')

,Age,CustomerDays,NKids,NTeens,NChildren,HasChild,Income,MntWines,PMntWines,MntFruits,PMntFruits,MntMeatProducts,PMntMeatProducts,MntFishProducts,PMntFishProducts,MntSweetProducts,PMntSweetProducts,MntGoldProds,PMntGoldProds,TotalMnt,Complain,NDealsPurchases,NWebPurchases,PWebPurchases,NCatalogPurchases,PCatalogPurchases,NStorePurchases,PStorePurchases,TotalPurchases,NWebVisitsMonth,Response1,Response2,Response3,Response4,Response5
Age,1.000000,-0.009413,-0.259770,0.379918,0.080665,-0.011792,0.215629,0.241767,0.292402,0.032190,-0.171249,0.117927,-0.185200,0.032706,-0.170872,-0.002000,-0.184172,0.080073,-0.175171,0.171360,-0.006009,0.076416,0.165379,-0.032703,0.182998,0.140828,0.176398,-0.107987,0.183693,-0.132688,-0.006097,0.006654,-0.075145,0.069500,-0.018985
CustomerDays,-0.009413,1.000000,-0.051099,0.020496,-0.022801,0.001520,-0.016453,0.161985,-0.009958,0.142152,0.042119,0.167695,-0.037042,0.141729,0.032048,0.128340,0.040611,0.236775,0.041598,0.185494,0.033084,0.219790,0.212221,0.107763,0.134935,0.080626,0.120261,-0.159284,0.170193,0.296640,-0.038751,0.009909,-0.010812,0.018943,-0.006836
NKids,-0.259770,-0.051099,1.000000,-0.037315,0.677724,0.541411,-0.526592,-0.578733,-0.128340,-0.440765,0.001133,-0.545048,0.134041,-0.442668,0.007413,-0.427744,-0.005729,-0.418838,0.309212,-0.609817,0.022985,0.257410,-0.421107,0.216254,-0.590776,-0.478323,-0.554991,0.278706,-0.578294,0.476764,-0.185553,-0.086638,0.018127,-0.163677,-0.212042
NTeens,0.379918,0.020496,-0.037315,1.000000,0.699888,0.608509,0.051310,0.111027,0.479248,-0.192226,-0.273884,-0.124173,-0.375571,-0.216358,-0.294717,-0.192520,-0.270903,-0.021277,-0.057393,-0.046701,0.001928,0.472194,0.140327,0.132504,-0.039063,-0.094644,0.073206,0.006058,0.039548,0.109812,-0.143047,-0.027155,-0.051451,0.044670,-0.192467
NChildren,0.080665,-0.022801,0.677724,0.699888,1.000000,0.856953,-0.347469,-0.332668,0.256064,-0.452197,-0.191542,-0.481805,-0.184287,-0.473267,-0.206374,-0.445707,-0.197523,-0.311486,0.182538,-0.471016,0.018516,0.532822,-0.187704,0.262353,-0.450109,-0.413589,-0.339605,0.198779,-0.379114,0.427970,-0.243904,-0.081210,-0.022968,-0.084741,-0.298768
HasChild,-0.011792,0.001520,0.541411,0.608509,0.856953,1.000000,-0.384978,-0.317873,0.226795,-0.421250,-0.140103,-0.473550,-0.229028,-0.448290,-0.161698,-0.411731,-0.147812,-0.270748,0.200247,-0.452833,0.006115,0.535075,-0.116606,0.313358,-0.431718,-0.419444,-0.299578,0.170834,-0.330278,0.462934,-0.276151,-0.089862,-0.013510,-0.075576,-0.341810
Income,0.215629,-0.016453,-0.526592,0.051310,-0.347469,-0.384978,1.000000,0.781647,0.217249,0.541785,-0.093591,0.768042,0.006375,0.541363,-0.124963,0.530352,-0.093590,0.474977,-0.504896,0.808514,-0.030034,-0.192669,0.531787,-0.237048,0.744149,0.590410,0.681090,-0.366556,0.728849,-0.613538,0.317170,0.096246,-0.019289,0.208756,0.370400
MntWines,0.241767,0.161985,-0.578733,0.111027,-0.332668,-0.317873,0.781647,1.000000,0.474296,0.515727,-0.227457,0.820524,-0.207970,0.522469,-0.254985,0.502275,-0.212699,0.581344,-0.547649,0.934322,-0.032579,0.061957,0.741818,-0.087934,0.822583,0.624157,0.804604,-0.505376,0.869805,-0.386012,0.307375,0.135981,0.036632,0.306858,0.360534
PMntWines,0.292402,-0.009958,-0.128340,0.479248,0.256064,0.226795,0.217249,0.474296,1.000000,-0.285808,-0.649150,-0.025093,-0.653079,-0.298900,-0.675525,-0.293070,-0.641176,-0.001962,-0.313737,0.178948,-0.030285,0.225792,0.268146,0.123141,0.121934,0.042630,0.202297,-0.121558,0.190542,0.067720,0.036175,0.094558,0.039826,0.262477,0.059814
MntFruits,0.032190,0.142152,-0.440765,-0.192226,-0.452197,-0.421250,0.541785,0.515727,-0.285808,1.000000,0.616710,0.714805,0.167787,0.708400,0.254476,0.694267,0.286392,0.572137,-0.214888,0.673303,-0.007043,-0.098489,0.473920,-0.156946,0.632412,0.512647,0.583004,-0.346840,0.628405,-0.434972,0.179019,0.009786,0.011690,0.027369,0.232181


<br>
In order to get a better overview of each correlation, the following cell is used. The function ``np.triu()`` takes the upper triangle of the matrix, removing duplicates and low-correlated pairs. It is then displayed the values of the highest correlation pairs.

In [29]:
# Reshapes the dataframe by converting the data into a stacked form
plcc_u = plcc.stack()

# Get the upper triangle of the matrix
tri_u = np.triu(np.ones(plcc.shape)).astype('bool').reshape(plcc.size)  

# Select unique pairs
plcc_u = plcc_u[tri_u]  

# Removes the low-correlated pairs and the auto-correlations, given a specified threshold
l_threshold = 0.7

u_threshold = 1
plcc_top = plcc_u[(plcc_u.abs() > l_threshold) & (plcc_u.abs() < u_threshold) & (plcc_u!=1.0)]

# Prints the output
print("Top most correlated feature pairs:\n", plcc_top.abs().sort_values(ascending=False))

Top most correlated feature pairs:
 MntMeatProducts    TotalMnt             0.942492
MntWines           TotalMnt             0.934322
NCatalogPurchases  PCatalogPurchases    0.912137
TotalMnt           TotalPurchases       0.910064
NStorePurchases    TotalPurchases       0.890645
TotalMnt           NCatalogPurchases    0.884669
NCatalogPurchases  TotalPurchases       0.875974
MntWines           TotalPurchases       0.869805
MntMeatProducts    TotalPurchases       0.864135
NChildren          HasChild             0.856953
MntMeatProducts    NCatalogPurchases    0.850073
NWebPurchases      TotalPurchases       0.830922
MntWines           NCatalogPurchases    0.822583
                   MntMeatProducts      0.820524
TotalMnt           NStorePurchases      0.812685
Income             TotalMnt             0.808514
MntWines           NStorePurchases      0.804604
Income             MntWines             0.781647
MntMeatProducts    NStorePurchases      0.777546
Income             MntMeatProduct

<br>

The values reached through the spearman's correlation provide important information about some features, for instance:

* The pairs MntMeatProducts and TotalMnt, MntWines and TotalMnt, are extremely highly correlated between each other. Those two pairs have the MntMeatProducts and MntWines, which are 'pieces of a whole', being that whole the TotalMnt, which implies that when a customer spends the most money in total, it also spends the most money in meat and wine. 
* The pair NCatalogPurchases and PCatalogPurchases, are also extremely highly correlated. This implies that when the number of catalog purchases increases, the amount of purchases using catalog instead of the other two channels, also increases.
* The pair TotalMnt and TotalPurchases, has, once again, an extremly high correlation. This entails that when the total number of purchases increases, the total amount of money spent also increases, which it is quite straightforward.
* There are a lot more pairs that are over the normal value of correlation, and therefore may create some redundancy in the model.

The strong correlations described are not beneficial for the learning environment of the model, therefore, they will have to cease to exist. To achieve that, it will fist be studied the discrimination of the features, so that the least relevant ones can be discarded and the redundancy issues can be solved.

<br>

### **6.** Exporting the dataset

In order to use this modified dataset in different notebooks, it is necessary to export it. 
The dataset is being exported in an excel format with the name "feature_extraction.xlsx". 

In [30]:
data.to_excel( "feature_extract.xlsx")